# Caffeine Form

Caffeine Form is a company creating coffee cups from recycled material.

Caffeine Form sells cups to coffee shops through their website. They would prefer to partner
directly with the shops.

The company believes that stores with more reviews will help them to better market their
product.

The company want to focus on the types of shop that get the most reviews.

They want you to investigate how types of shop and number of reviews are related.

# Task 1

Before you start your analysis, you will need to make sure the data is clean. 

The table below shows what the data should look like. 

Create a cleaned version of the dataframe. 

 - You should start with the data in the table `coffee`. 

 - All column names and values should match the table below.

| Column Name | Criteria                                                |
|-------------|---------------------------------------------------------|
| Region    | Nominal. </br> Where the store is located. One of 10 possible regions (A to J).</br> Missing values should be replaced with “Unknown”.|
| Place name | Nominal. </br>The name of the store. </br>Missing values should be replaced with “Unknown”.|
| Place type  | Nominal. </br>The type of coffee shop. One of “Coffee shop”, “Cafe”, “Espresso bar”, and “Others”. </br>Missing values should be replaced with “Unknown”. |
| Rating   | Ordinal. </br>Average rating of the store from reviews. On a 5 point scale. </br>Missing values should be replaced with 0. |
| Reviews  | Nominal. </br>The number of reviews given to the store. </br>Missing values should be replaced with the overall median number, rounded to the nearest interger value.|
| Price  | Ordinal. </br>The price range of products in the store. One of '\$', '\$\$' or '\$\$\$'. </br>Missing values should be replaced with ”Unknown”.|
| Delivery option   | Nominal. </br>If delivery is available. Either True or False. </br>Missing values should be replaced with False. |
| Dine in option | Nominal. </br>If dine in is available. Either True or False. </br>Missing values should be replaced with False. |
| Takeout option | Nominal. </br>If take away is available. Either True or False. </br>Missing values should be replaced with False.|

In [6]:
select *
from coffee

,Region,Place name,Place type,Rating,Reviews,Price,Delivery option,Dine in option,Takeout option
0,C,Dim Kavu,Others,4.6,206.0,$$,False,NaN,NaN
1,C,Коферум,Cafe,5.0,24.0,$$,False,NaN,1.0
2,C,Кофейня Світ Чаю,Coffee shop,5.0,11.0,$$,False,NaN,1.0
3,C,Кофейня Starcoff,Coffee shop,4.4,331.0,$$,False,1.0,1.0
4,C,"Кофейня ""Friend Zone""",Coffee shop,5.0,12.0,$$,False,1.0,1.0
...,...,...,...,...,...,...,...,...,...
195,H,Artist,Others,4.6,963.0,$$$,False,1.0,1.0
196,H,Dream Cafe,Cafe,4.3,646.0,$$,False,1.0,1.0
197,H,РУТА КАФЕ ПП КУЛІЧЕНКО К.І.,Cafe,4.6,16.0,$$,False,1.0,1.0
198,H,Ob'yektna Kava,Coffee shop,4.7,599.0,$$,False,1.0,1.0


In [8]:
WITH non_null_data AS(
	SELECT
		-- Fill nulls with the provided value --
		COALESCE("Region", 'Unknown') AS "Region",
		COALESCE("Place name", 'Unknown') AS "Place name",
	    COALESCE("Place type", 'Unknown') AS "Place type",
	    COALESCE("Rating", '0') AS "Rating",
  		COALESCE("Reviews",percentile_cont(0.5) WITHIN GROUP (ORDER BY "Reviews")) AS "Reviews",
	    COALESCE("Price", 'Unknown') AS "Price",
  	    COALESCE("Delivery option", 'False') AS "Delivery option", 
  	    COALESCE("Dine in option", 'False') AS "Dine in option",
  	    COALESCE("Takeout option", 'False') AS "Takeout option",
		-- Check the duplicate data --
		COUNT(*) AS duplicate
	FROM coffee
	GROUP BY "Region",
			 "Place name",
			 "Place type",
			 "Rating",
			 "Reviews",
			 "Price",
			 "Delivery option",
			 "Dine in option",
			 "Takeout option"),
	--- Eliminate column with all service option values are false ---
	false_value AS (
	SELECT *
	FROM non_null_data
	WHERE "Place name" NOT IN (SELECT "Place name"
						 	  From non_null_data
							  WHERE "Delivery option" = 'False' AND
						  			"Dine in option" = 'False' AND
						  	 		"Takeout option" = 'False')),
	-- Fill duplicates data with the average  ---
	duplicate_check AS(
	SELECT 
	   "Region",
	   "Place name",
	   "Place type",
	   CASE WHEN duplicate > 1 THEN AVG("Rating")
	        ELSE "Rating"
	   END AS "Rating",
	   CASE WHEN duplicate > 1 THEN AVG("Reviews")
	   	    ELSE "Reviews"
	   END AS "Reviews",
	   "Price",
	   "Delivery option",
	   "Dine in option",
	   "Takeout option"
	FROM false_value
	GROUP BY "Region",
		     "Place name",
		 	 "Place type",
			 "Rating",
			 "Reviews",
			 "Price",
			 "Delivery option",
			 "Dine in option",
			 "Takeout option",
			 duplicate)

SELECT *
FROM duplicate_check;

,Region,Place name,Place type,Rating,Reviews,Price,Delivery option,Dine in option,Takeout option
0,C,Veterano Coffee,Coffee shop,4.5,136,$$,False,False,True
1,E,Fixage,Coffee shop,4.6,702,$$,False,True,True
2,H,Artist,Others,4.6,963,$$$,False,True,True
3,G,Kaffa,Espresso bar,4.8,824,$$,False,True,True
4,C,Кофейня Starcoff,Coffee shop,4.4,331,$$,False,True,True
...,...,...,...,...,...,...,...,...,...
160,C,Займемся Кофе,Coffee shop,4.6,200,$$,False,True,True
161,G,Matílda cafe,Cafe,4.9,26,$$,False,True,True
162,H,Zeffirino,Coffee shop,4.6,830,$$,False,True,True
163,H,Бамбук,Cafe,4.1,189,$$,False,True,True


# Task 2 

The team at Caffine Form believe that the number of reviews changes depending on the type of store. 

Producing a table showing the difference in the median number of reviews by rating along with the minimum and maximum number of reviews to investigate this question for the team.

 - You should start with the data in the table `coffee`.

 - It should include the three columns `Place type`, `min_review`, `max_review`. 

In [10]:
WITH non_null_data AS(
	SELECT
		-- Fill nulls with the provided value --
		COALESCE("Region", 'Unknown') AS "Region",
		COALESCE("Place name", 'Unknown') AS "Place name",
	    COALESCE("Place type", 'Unknown') AS "Place type",
	    COALESCE("Rating", '0') AS "Rating",
  		COALESCE("Reviews",percentile_cont(0.5) WITHIN GROUP (ORDER BY "Reviews")) AS "Reviews",
	    COALESCE("Price", 'Unknown') AS "Price",
  	    COALESCE("Delivery option", 'False') AS "Delivery option", 
  	    COALESCE("Dine in option", 'False') AS "Dine in option",
  	    COALESCE("Takeout option", 'False') AS "Takeout option",
		-- Check the duplicate data --
		COUNT(*) AS duplicate
	FROM coffee
	GROUP BY "Region",
			 "Place name",
			 "Place type",
			 "Rating",
			 "Reviews",
			 "Price",
			 "Delivery option",
			 "Dine in option",
			 "Takeout option"),
	--- Eliminate column with all service option values are false ---
	false_value AS (
	SELECT *
	FROM non_null_data
	WHERE "Place name" NOT IN (SELECT "Place name"
						 	  From non_null_data
							  WHERE "Delivery option" = 'False' AND
						  			"Dine in option" = 'False' AND
						  	 		"Takeout option" = 'False')),
	-- Fill duplicates data with the average  ---
	duplicate_check AS(
	SELECT 
	   "Region",
	   "Place name",
	   "Place type",
	   CASE WHEN duplicate > 1 THEN AVG("Rating")
	        ELSE "Rating"
	   END AS "Rating",
	   CASE WHEN duplicate > 1 THEN AVG("Reviews")
	   	    ELSE "Reviews"
	   END AS "Reviews",
	   "Price",
	   "Delivery option",
	   "Dine in option",
	   "Takeout option"
	FROM false_value
	GROUP BY "Region",
		     "Place name",
		 	 "Place type",
			 "Rating",
			 "Reviews",
			 "Price",
			 "Delivery option",
			 "Dine in option",
			 "Takeout option",
			 duplicate),
	-- Rank data by place type --
	RankedReviews AS (
    SELECT
        "Place type",
        ROUND("Rating"::INTEGER, 0) AS "Rating",
        "Reviews",
        ROW_NUMBER() OVER (PARTITION BY "Place type", ROUND("Rating"::INTEGER, 0) ORDER BY "Reviews") AS RowNum,
        COUNT("Reviews") OVER (PARTITION BY "Place type", ROUND("Rating"::INTEGER, 0)) AS TotalRows,
		SUM("Reviews") OVER(PARTITION BY "Place type") AS total_reviews
    FROM duplicate_check
	WHERE "Rating" > 0)
	
SELECT
    "Place type",
    "Rating",
    COALESCE(AVG("Reviews"), 0) AS median_reviews,
    MIN("Reviews") AS min_reviews,
    MAX("Reviews") AS max_reviews,
	total_reviews
FROM RankedReviews
WHERE RowNum BETWEEN (TotalRows / 2) + 1 AND (TotalRows / 2) + 2 OR TotalRows % 2 = 1 -- Find median of the data --
GROUP BY "Place type", "Rating", total_reviews
ORDER BY total_reviews DESC, "Place type", median_reviews DESC;

,Place type,Rating,median_reviews,min_reviews,max_reviews,total_reviews
0,Coffee shop,5,886.646154,10,17937,69376
1,Coffee shop,4,646.000000,593,699,69376
2,Cafe,4,726.000000,646,806,29910
3,Cafe,5,164.500000,157,172,29910
4,Others,4,1345.000000,1345,1345,10727
5,Others,5,682.000000,401,963,10727
6,Espresso bar,5,742.384615,14,1908,9678
7,Espresso bar,4,27.000000,27,27,9678


The presented summary table distinguishes between two rating categories: a 5-rating designation is assigned to data falling within the range of 4.5 to 5, while a 4-rating designation pertains to data falling within the range of 4.0 to 4.4. Notably, coffee shops garner the highest number of reviews, accumulating a total of 69,376 reviews and constituting 50% of the overall review count. It is noteworthy that the maximum number of reviews is not correlated with place type ratings. This is evident in the observation that coffee shops and espresso bars with a rating of 5 boast the highest reviews in their category, whereas cafes and other place types with a rating of 4 achieve the maximum reviews within their respective category. The following is a recommendation of the top 10 coffee shops with the highest reviews for potential partnership opportunities with Coffee From:

In [6]:
WITH non_null_data AS(
	SELECT
		-- Fill nulls with the provided value --
		COALESCE("Region", 'Unknown') AS "Region",
		COALESCE("Place name", 'Unknown') AS "Place name",
	    COALESCE("Place type", 'Unknown') AS "Place type",
	    COALESCE("Rating", '0') AS "Rating",
  		COALESCE("Reviews",percentile_cont(0.5) WITHIN GROUP (ORDER BY "Reviews")) AS "Reviews",
	    COALESCE("Price", 'Unknown') AS "Price",
  	    COALESCE("Delivery option", 'False') AS "Delivery option", 
  	    COALESCE("Dine in option", 'False') AS "Dine in option",
  	    COALESCE("Takeout option", 'False') AS "Takeout option",
		-- Check the duplicate data --
		COUNT(*) AS duplicate
	FROM coffee
	GROUP BY "Region",
			 "Place name",
			 "Place type",
			 "Rating",
			 "Reviews",
			 "Price",
			 "Delivery option",
			 "Dine in option",
			 "Takeout option"),
	--- Eliminate column with all service option values are false ---
	false_value AS (
	SELECT *
	FROM non_null_data
	WHERE "Place name" NOT IN (SELECT "Place name"
						 	  From non_null_data
							  WHERE "Delivery option" = 'False' AND
						  			"Dine in option" = 'False' AND
						  	 		"Takeout option" = 'False')),
	-- Fill duplicates data with the average  ---
	duplicate_check AS(
	SELECT 
	   "Region",
	   "Place name",
	   "Place type",
	   CASE WHEN duplicate > 1 THEN AVG("Rating")
	        ELSE "Rating"
	   END AS "Rating",
	   CASE WHEN duplicate > 1 THEN AVG("Reviews")
	   	    ELSE "Reviews"
	   END AS "Reviews",
	   "Price",
	   "Delivery option",
	   "Dine in option",
	   "Takeout option"
	FROM false_value
	GROUP BY "Region",
		     "Place name",
		 	 "Place type",
			 "Rating",
			 "Reviews",
			 "Price",
			 "Delivery option",
			 "Dine in option",
			 "Takeout option",
			 duplicate)
		
SELECT *
FROM duplicate_check
WHERE "Place type" = 'Coffee shop' AND 
	  "Rating" > 4.4
ORDER BY "Reviews" DESC
LIMIT 10;

,Region,Place name,Place type,Rating,Reviews,Price,Delivery option,Dine in option,Takeout option
0,E,Lviv Coffee Manufacture,Coffee shop,4.7,17937,$$,True,True,True
1,E,Svit Kavy,Coffee shop,4.6,2931,$$,False,True,True
2,I,Kofeyin,Coffee shop,4.5,2914,$$,False,True,True
3,E,Вірменка,Coffee shop,4.8,2873,$$,False,True,True
4,A,Traveler`s Coffee,Coffee shop,4.6,2700,$$,True,True,True
5,G,ONE LOVE espresso bar,Coffee shop,4.6,2319,$$,False,True,True
6,A,Zheto,Coffee shop,4.8,2141,$$,False,True,True
7,E,Na bambetli,Coffee shop,4.7,2089,$$,False,True,True
8,H,Try Bobry,Coffee shop,4.6,2013,$$,False,True,True
9,J,Kavun,Coffee shop,4.7,1785,$$,True,True,False
